In [1]:
# pip install faster-whisper

In [2]:
# pip install keyboard


In [3]:
# pip install sounddevice

In [4]:
import sounddevice as sd
import numpy as np
import keyboard
import wave
import tempfile
from faster_whisper import WhisperModel

In [5]:
def record_audio(sample_rate=16000):
    print("Recording... Press any key to stop.")
    audio_data = []
    
    # Callback function to append audio data when available
    def callback(indata, frames, time, status):
        if status:
            print(status)
        audio_data.append(indata.copy())
    
    # Start the audio stream
    with sd.InputStream(samplerate=sample_rate, channels=1, dtype='float32', callback=callback):
        keyboard.wait('enter')  # Wait for any key press to stop recording
    
    # Concatenate recorded audio data
    audio_data = np.concatenate(audio_data)
    print("Recording stopped.")
    
    # Create a temporary WAV file to save the recorded audio
    with tempfile.NamedTemporaryFile(suffix=".wav", delete=False) as temp_file:
        wf = wave.open(temp_file.name, 'wb')
        wf.setnchannels(1)
        wf.setsampwidth(2)
        wf.setframerate(sample_rate)
        wf.writeframes((audio_data * np.iinfo(np.int16).max).astype(np.int16).tobytes())
        wf.close()

        return temp_file.name, sample_rate

In [6]:
model_size = "medium"  # medium is better 
model = WhisperModel(model_size, device="auto", compute_type="float16")

In [26]:
audio_path, sample_rate = record_audio() # records audio from mic
beam_size = 5  # You can adjust the beam size as needed
segments, info = model.transcribe(audio_path, beam_size=beam_size)
for segment in segments:
    print("[%.2fs -> %.2fs] %s" % (segment.start, segment.end, segment.text))

Recording... Press any key to stop.


PortAudioError: Error querying device -1